In [166]:
import numpy as np
import random
import queue
arr = np.genfromtxt('Day15.txt',delimiter = ',',dtype=np.int64)

In [17]:
def indexing(listt,index):
    inxlist = []
    global arr1
    for i, ss in enumerate(listt):
        if ss == '0': #position mode
            inxlist.append(arr1[index+i+1])
        elif ss =='1': #immediate mode
            inxlist.append(index+i+1)
        else: #ss == '2'
            global base
            inxlist.append(arr1[index+i+1]+base)
    return inxlist

def arrOdict(inxlist):
    global arr1
    global lgmem
    vallist = []
    for inx in inxlist:
        if inx > (len(arr1)-1):
            if inx not in lgmem.keys():
                lgmem[inx] = 0
                vallist.append(lgmem[inx])
            else:
                vallist.append(lgmem[inx])
        else:
            vallist.append(arr1[inx])
    return vallist

def setequal(inxequal,num):
    global arr1
    global lgmem
    if inxequal > (len(arr1)-1):
        lgmem[inxequal] = num
    else:
        arr1[inxequal] = num
    return None

def opcodeAct(index,inxlist,opcode):
    global arr1
    global inpt
    global statusQ
    vallist = arrOdict(inxlist)
    if opcode == 1:
        setequal(inxlist[2],vallist[0] + vallist[1])
        index += 4
    elif opcode == 2:
        setequal(inxlist[2],vallist[0] * vallist[1])
        index += 4
    elif opcode == 4:
        statusQ.put(vallist[0])
        index += 2
    elif opcode == 3:
        setequal(inxlist[0],inpt)
        index += 2
    elif opcode == 5:
        if vallist[0]: #If non-zero is true
            index = vallist[1]
        else:
            index += 3
    elif opcode == 6:
        if not vallist[0]:
            index = vallist[1]
        else:
            index += 3
    elif opcode == 7:
        if vallist[0] < vallist[1]:
            setequal(inxlist[2],1)
        else:
            setequal(inxlist[2],0)
        index += 4
    elif opcode == 8:
        if vallist[0] == vallist[1]:
            setequal(inxlist[2],1)
        else:
            setequal(inxlist[2],0)
        index += 4
    elif opcode == 9:
        global base
        base += vallist[0]
        index += 2
    return index

def day9(arr):
    global arr1
    global inpt
    global coord
    global statusQ
    arr1 = arr.copy()
    index = 0
    statusQ = queue.Queue()
    status = 0
    curxy = [0,0]
    coord[tuple(curxy)] = 1
    while status != 2:
        direc = [1,2,3,4]
        curdir = random.choice(direc)
        inpt = curdir
        status = 0
        while status == 0:
            if arr1[index] in [1,2,7,8]:
                index = opcodeAct(index,arr1[index+1:index+4],arr1[index])
            elif arr1[index] in [5,6]:
                index = opcodeAct(index,arr1[index+1:index+3],arr1[index])
            elif arr1[index] in [3,4,9]:
                index = opcodeAct(index,arr1[index+1:index+2],arr1[index])
            else: #5 digit opcode
                s = str(arr1[index])
                s = s.rjust(5,'0')
                s = s[::-1]
                if int(s[0]) in [1,2,7,8]:
                    inxlist = indexing(s[2:],index)
                elif int(s[0]) in [5,6]:
                    inxlist = indexing(s[2:4],index)
                elif int(s[0]) in [3,4,9]:
                    inxlist = indexing(s[2:3],index)
                index = opcodeAct(index,inxlist,int(s[0]))
            if len(statusQ.queue) == 1:
                status = statusQ.get()
                if status !=0:
                    if curdir == 1:
                        curxy[1] += 1 
                    elif curdir == 2:
                        curxy[1] -= 1
                    elif curdir == 3:
                        curxy[0] -= 1
                    else:
                        curxy[0] += 1
                    coord[tuple(curxy)] = status
                else:
                    direc.remove(curdir)
                    curdir = random.choice(direc)
                    inpt = curdir
    return None

In [18]:
base = 0
lgmem = {}
coord = {}
inpt = 0
day9(arr)

In [137]:
coord_arr = np.array([[keys[0][0],keys[0][1],keys[1]] for keys in coord.items()],dtype=np.int)
arrlen_p = np.max(np.max(abs(coord_arr[:,0:2]),axis=0))+2
coord_arr[:,0:2] += arrlen_p
arrmat_p = np.zeros(shape=(arrlen_p*2,arrlen_p*2),dtype=np.int)

for row in coord_arr:
    arrmat_p[row[0]][row[1]] = row[2]
arrmat_p[arrlen_p,arrlen_p] = 3

In [139]:
a = 1
while a !=0:
    for i in range(1,arrmat_p.shape[0]):
        for j in range(1,arrmat_p.shape[1]):
            if arrmat_p[i][j] == 1:
                sur = arrmat_p[i+1][j] + arrmat_p[i-1][j] + arrmat_p[i][j-1] + arrmat_p[i][j+1]
                if sur == 1:
                    arrmat_p[i][j] = 0
    a=0
    for i in range(1,arrmat_p.shape[0]):
        for j in range(1,arrmat_p.shape[1]):
            if arrmat_p[i][j] == 1:
                sur = arrmat_p[i+1][j] + arrmat_p[i-1][j] + arrmat_p[i][j-1] + arrmat_p[i][j+1]
                if sur == 1:
                    a=1
                    break
np.sum(arrmat_p == 1)+1